# Do interface design on the best docks

### Boilerplate

In [1]:
%load_ext lab_black
# python internal
import collections
import copy
import gc
from glob import glob
import h5py
import itertools
import os
import random
import re
import socket
import shutil
import subprocess
import sys

# conda/pip
import dask
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
from tqdm import tqdm

# special packages on the DIGS
import py3Dmol
import pymol
import pyrosetta

# notebook magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

print(os.getcwd())
print(socket.gethostname())

/mnt/home/pleung/projects/peptide_binders/r0/peptide_binders
dig65


### Make a function to do one sided PPI design
use `tutorial_design_v5.xml` as an example
added CYS/CYD restriction  
try `<Index name="" resnums="{}" />` instead of slices             
```
<Slice name="patchdock_res" indices="{interface_hydrophobics}" selector="chainB" />
<Slice name="pocket" slice_mode="CONTIGUOUS_OR" selector="chainB" indices="{interface_hydrophobics}" />
```

In [11]:
from pyrosetta.distributed.packed_pose.core import PackedPose
from pyrosetta.distributed import requires_init


@requires_init
def design_binder(packed_pose_in: PackedPose, **kwargs) -> PackedPose:
    """
    TODO
    """
    import bz2
    import pyrosetta
    from pyrosetta.rosetta.core.pose import Pose
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed.tasks.rosetta_scripts import (
        SingleoutputRosettaScriptsTask,
    )

    if packed_pose_in == None:
        file = kwargs["-s"]
        with open(file, "rb") as f:
            packed_pose_in = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())
        scores = pyrosetta.distributed.cluster.get_scores_dict(file)["scores"]
    else:
        raise RuntimeError("Need to supply an input")

    if "sasa_threshold" in kwargs.keys():
        sasa_threshold = kwargs["sasa_threshold"]
    else:
        sasa_threshold = 1350

    print(scores["interface_hydrophobics"])  # TODO
    print(scores["target_name"])  # TODO
    xml = """
    <ROSETTASCRIPTS>
        <SCOREFXNS>
            <ScoreFunction name="sfxn" weights="beta_nov16" />
            <ScoreFunction name="sfxn_soft" weights="beta_nov16_soft" />
            <ScoreFunction name="sfxn_fa_atr" weights="empty" >
                <Reweight scoretype="fa_atr" weight="1" />
            </ScoreFunction>
            <ScoreFunction name="sfxn_relax" weights="beta_nov16" >
                <Reweight scoretype="arg_cation_pi" weight="3" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="5" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-0.5" />
            </ScoreFunction>
            <ScoreFunction name="sfxn_design" weights="beta_nov16" >
                <Reweight scoretype="res_type_constraint" weight="1.5" />
                <Reweight scoretype="aa_composition" weight="1.0" />
                <Reweight scoretype="arg_cation_pi" weight="3" />
                <Reweight scoretype="approximate_buried_unsat_penalty" weight="5" />
                <Set approximate_buried_unsat_penalty_burial_atomic_depth="3.5" />
                <Set approximate_buried_unsat_penalty_hbond_energy_threshold="-0.5" />
                <Set approximate_buried_unsat_penalty_hbond_bonus_cross_chain="-1" />
            </ScoreFunction>
            <ScoreFunction name="vdw_sol" weights="empty" >
                <Reweight scoretype="fa_atr" weight="1.0" />
                <Reweight scoretype="fa_rep" weight="0.55" />
                <Reweight scoretype="fa_sol" weight="1.0" />
            </ScoreFunction>
        </SCOREFXNS>
        <TASKOPERATIONS>
            <SelectBySASA name="PR_monomer_core_sel" mode="sc" state="monomer" probe_radius="2.2" core_asa="15" surface_asa="15" core="0" boundary="1" surface="1" verbose="0" />
        </TASKOPERATIONS>
        <RESIDUE_SELECTORS>
            <Chain name="chainA" chains="A"/>
            <Chain name="chainB" chains="B"/>
            <Neighborhood name="interface_chA" selector="chainB" distance="10.0" />
            <Neighborhood name="interface_chB" selector="chainA" distance="10.0" />
            <And name="AB_interface" selectors="interface_chA,interface_chB" />
            <Not name="Not_interface" selector="AB_interface" />
            <And name="actual_interface_chA" selectors="AB_interface,chainA" />
            <And name="actual_interface_chB" selectors="AB_interface,chainB" />
            <And name="chainB_not_interface" selectors="Not_interface,chainB" />

            <ResidueName name="pro_and_gly_positions" residue_name3="PRO,GLY,CYS,CYD" />
            <ResidueName name="apolar" residue_name3="ALA,CYS,PHE,ILE,LEU,MET,THR,PRO,VAL,TRP,TYR" />
            <Not name="polar" selector="apolar" />

            <InterfaceByVector name="interface_by_vector" cb_dist_cut="11" nearby_atom_cut="5.5" vector_angle_cut="75" vector_dist_cut="9" grp1_selector="actual_interface_chA" grp2_selector="actual_interface_chB"/>
            <Task name="all_cores" fixed="true" task_operations="PR_monomer_core_sel" packable="false" designable="false"/>
            <And name="for_hydrophobic" selectors="actual_interface_chA,interface_by_vector">
                <Not selector="all_cores" />
            </And>

            <ResiduePDBInfoHasLabel name="HOTSPOT_res" property="HOTSPOT" />
            <ResiduePDBInfoHasLabel name="DONE_res" property="DONE" />
            
            <Index name="res1" resnums="1" />
            <Index name="patchdock_res" resnums="{interface_hydrophobics}" />

            <Index name="pocket" resnums="{interface_hydrophobics}" />
            <And name="target_not_pocket" selectors="chainB">
                <Not selector="pocket" />
            </And>

            <Layer name="surface" select_core="false" select_boundary="false" select_surface="true" use_sidechain_neighbors="true"/>
            <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" use_sidechain_neighbors="true"/>
            <Layer name="core" select_core="true" select_boundary="false" select_surface="false" use_sidechain_neighbors="true"/>
            <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
            <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
            <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
            <And name="helix_cap" selectors="entire_loop">
                <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
            </And>
            <And name="helix_start" selectors="entire_helix">
                <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
            </And>
            <And name="helix" selectors="entire_helix">
                <Not selector="helix_start"/>
            </And>
            <And name="loop" selectors="entire_loop">
                <Not selector="helix_cap"/>
            </And>

        </RESIDUE_SELECTORS>
        <TASKOPERATIONS>
            <DesignRestrictions name="layer_design_F_boundary_M">
                <Action selector_logic="surface AND helix_start"  aas="DEHKPQR"/>
                <Action selector_logic="surface AND helix"        aas="EHKQR"/>
                <Action selector_logic="surface AND sheet"        aas="EHKNQRST"/>
                <Action selector_logic="surface AND loop"         aas="DEGHKNPQRST"/>
                <Action selector_logic="boundary AND helix_start" aas="ADEFHIKLMNPQRSTVWY"/>
                <Action selector_logic="boundary AND helix"       aas="ADEFHIKLMNQRSTVWY"/>
                <Action selector_logic="boundary AND sheet"       aas="DEFHIKLMNQRSTVWY"/>
                <Action selector_logic="boundary AND loop"        aas="ADEFGHIKLNPQRSTVWY"/>
                <Action selector_logic="core AND helix_start"     aas="AFILMPVWY"/>
                <Action selector_logic="core AND helix"           aas="AFILMVWYDENQSTH"/>
                <Action selector_logic="core AND sheet"           aas="FILMVWYDENQSTH"/>
                <Action selector_logic="core AND loop"            aas="AFGILPVWYDENQSTH"/>
                <Action selector_logic="helix_cap"                aas="DNST"/>
            </DesignRestrictions>
        </TASKOPERATIONS>
        <MOVERS>
            <AddCompositionConstraintMover name="3trp" >
                <Comp entry="PENALTY_DEFINITION;TYPE TRP;ABSOLUTE 0;PENALTIES 0 3;DELTA_START 0;DELTA_END 1;BEFORE_FUNCTION CONSTANT;AFTER_FUNCTION LINEAR;END_PENALTY_DEFINITION;" />
            </AddCompositionConstraintMover>
            <AddCompositionConstraintMover name="2met" >
                <Comp entry="PENALTY_DEFINITION;TYPE MET;ABSOLUTE 0;PENALTIES 0 2;DELTA_START 0;DELTA_END 1;BEFORE_FUNCTION CONSTANT;AFTER_FUNCTION LINEAR;END_PENALTY_DEFINITION;" />
            </AddCompositionConstraintMover>
            <AddCompositionConstraintMover name="30_percent_polar" selector="for_hydrophobic">
                <Comp entry="PENALTY_DEFINITION;TYPE ASP GLU HIS LYS ASN GLN ARG SER THR TYR;FRACT_DELTA_START -0.01;FRACT_DELTA_END 0.0;PENALTIES 0.1 0 ;FRACTION 0.30;BEFORE_FUNCTION QUADRATIC;AFTER_FUNCTION CONSTANT;END_PENALTY_DEFINITION" />
            </AddCompositionConstraintMover>
        </MOVERS>
        <TASKOPERATIONS>
            <PruneBuriedUnsats name="prune_buried_unsats" allow_even_trades="false" atomic_depth_cutoff="3.5" minimum_hbond_energy="-0.5" />
            <ProteinProteinInterfaceUpweighter name="upweight_interface" interface_weight="3" />
            <ProteinInterfaceDesign name="pack_long" design_chain1="0" design_chain2="0" jump="1" interface_distance_cutoff="15"/>
            <IncludeCurrent name="current" />
            <LimitAromaChi2 name="limitchi2" chi2max="110" chi2min="70" include_trp="True" />
            <ExtraRotamersGeneric name="ex1_ex2" ex1="1" ex2aro="1" />

            <OperateOnResidueSubset name="restrict_target_not_interface" selector="chainB_not_interface">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="restrict_to_interface" selector="Not_interface">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="restrict_target2repacking" selector="chainB">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <OperateOnResidueSubset name="restrict_hotspots2repacking" selector="HOTSPOT_res">
                <RestrictToRepackingRLT/>
            </OperateOnResidueSubset>
            <DisallowIfNonnative name="disallow_GLY" resnum="0" disallow_aas="G" />
            <DisallowIfNonnative name="disallow_PRO" resnum="0" disallow_aas="P" />
            <OperateOnResidueSubset name="restrict_PRO_GLY" selector="pro_and_gly_positions">
                <PreventRepackingRLT/>
            </OperateOnResidueSubset>

            <SelectBySASA name="PR_monomer_core" mode="sc" state="monomer" probe_radius="2.2" core_asa="10" surface_asa="10" core="0" boundary="1" surface="1" verbose="0" />

        </TASKOPERATIONS>
        <RESIDUE_SELECTORS>
            <And name="sheetA" selectors="sheet,chainA" />
        </RESIDUE_SELECTORS>
        <MOVERS>

            <DeleteRegionMover name="delete_target_not_pocket" residue_selector="target_not_pocket" rechain="false" />

            <SwitchChainOrder name="chain1onlypre" chain_order="1" />
            <ScoreMover name="scorepose" scorefxn="sfxn" verbose="false" />
            <ParsedProtocol name="chain1only">
                <Add mover="chain1onlypre" />
                <Add mover="scorepose" />
            </ParsedProtocol>
            <TaskAwareMinMover name="min" scorefxn="sfxn" bb="0" chi="1" task_operations="pack_long" />

            <DeleteRegionMover name="delete_polar" residue_selector="polar" rechain="false" />

            <FastRelax name="relax_chain1" scorefxn="sfxn" repeats="1" batch="false" ramp_down_constraints="false" cartesian="false" bondangle="false" bondlength="false" min_type="dfpmin_armijo_nonmonotone" task_operations="ex1_ex2,limitchi2" >
                <MoveMap name="MM" >
                    <Chain number="1" chi="true" bb="true" />
                </MoveMap>
            </FastRelax>

            <ParsedProtocol name="chain1only_relax">
                <Add mover="chain1onlypre" />
                <Add mover="relax_chain1" />
            </ParsedProtocol>

            <StructProfileMover name="genProfile" add_csts_to_pose="1" consider_topN_frags="100" eliminate_background="0" ignore_terminal_residue="1" only_loops="0" burialWt="0" RMSthreshold="0.6" residue_selector="chainA" />

            <ClearConstraintsMover name="clear_constraints" />

            <SavePoseMover name="save_output" restore_pose="0" reference_name="pose_output" />
            <SavePoseMover name="load_output" restore_pose="1" reference_name="pose_output" />

            <LabelPoseFromResidueSelectorMover name="remove_done" remove_property="DONE"  residue_selector="res1" />
            <LabelPoseFromResidueSelectorMover name="add_done" property="DONE" residue_selector="res1" />

        </MOVERS>
        <FILTERS>
            <Sasa name="interface_buried_sasa" confidence="0" />
            <Sasa name="gate_filter" threshold="{sasa_threshold}" confidence="1" />
            <Ddg name="ddg"  threshold="-10" jump="1" repeats="5" repack="1" relax_mover="min" confidence="0" scorefxn="sfxn" extreme_value_removal="1" />

            <ShapeComplementarity name="interface_sc" verbose="0" min_sc="0.55" write_int_area="1" write_median_dist="1" jump="1" confidence="0"/>
            <ShapeComplementarity name="interface_sc_for_fr1" verbose="0" min_sc="0.55" write_int_area="1" write_median_dist="1" jump="1" confidence="0"/>

            <ScoreType name="total_score_MBF" scorefxn="sfxn" score_type="total_score" threshold="0" confidence="0" />
            <MoveBeforeFilter name="total_score_monomer" mover="chain1only" filter="total_score_MBF" confidence="0" />
            <ResidueCount name="res_count_MBF" max_residue_count="9999" confidence="0"/>
            <MoveBeforeFilter name="res_count_monomer" mover="chain1only" filter="res_count_MBF" confidence="0" />

            <CalculatorFilter name="score_per_res" equation="total_score_monomer / res" threshold="-3.5" confidence="0">
                <Var name="total_score_monomer" filter="total_score_monomer"/>
                <Var name="res" filter="res_count_monomer"/>
            </CalculatorFilter>

            <MoveBeforeFilter name="total_score_relax_monomer" mover="chain1only_relax" filter="total_score_MBF" confidence="0" />

            <CalculatorFilter name="score_per_res_relax" equation="total_score_relax_monomer / res" threshold="-3.5" confidence="0">
                <Var name="total_score_relax_monomer" filter="total_score_relax_monomer"/>
                <Var name="res" filter="res_count_monomer"/>
            </CalculatorFilter>

            <BuriedUnsatHbonds name="buns_heavy_ball_1.1D" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="AB_interface" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" burial_cutoff="0.01" dalphaball_sasa="true" probe_radius="1.1" max_hbond_energy="1.5" burial_cutoff_apo="0.2" />
            <BuriedUnsatHbonds name="sbuns5.0_heavy_ball_1.1D" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="AB_interface" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" burial_cutoff="0.01" dalphaball_sasa="true" probe_radius="1.1" atomic_depth_selection="5.0" atomic_depth_deeper_than="false" burial_cutoff_apo="0.2" atomic_depth_resolution="0.49" max_hbond_energy="1.5"/>

            <BuriedUnsatHbonds name="vbuns5.0_heavy_ball_1.1D" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="AB_interface" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" dalphaball_sasa="true" probe_radius="1.1" atomic_depth_selection="5.0" burial_cutoff="1000" burial_cutoff_apo="0.2" atomic_depth_apo_surface="5.5" atomic_depth_resolution="0.49" max_hbond_energy="1.5"/>

            <InterfaceHydrophobicResidueContacts name="hydrophobic_residue_contacts" target_selector="chainB" binder_selector="chainA" scorefxn="sfxn_soft" confidence="0"/> 
            <ContactMolecularSurface name="contact_molecular_surface" distance_weight="0.5" target_selector="chainA" binder_selector="chainB" confidence="0" />
            <ContactMolecularSurface name="contact_patch" distance_weight="0.5" target_selector="patchdock_res" binder_selector="chainA" confidence="0" />

            <Ddg name="ddg_hydrophobic_pre"  threshold="-10" jump="1" repeats="1" repack="0" confidence="0" scorefxn="vdw_sol" />
            <MoveBeforeFilter name="ddg_hydrophobic" mover="delete_polar" filter="ddg_hydrophobic_pre" confidence="0"/>

            <Ddg name="fa_atr_pocket_pre"  threshold="-15" jump="1" repeats="1" repack="0" confidence="0" scorefxn="sfxn_fa_atr" />
            <MoveBeforeFilter name="fa_atr_pocket" mover="delete_target_not_pocket" filter="fa_atr_pocket_pre" confidence="0"/>

            <SSPrediction name="pre_sspred_overall" cmd="/software/psipred4/runpsipred_single" use_probability="0" use_svm="0" threshold="0.85" confidence="0" />
            <MoveBeforeFilter name="sspred_overall" mover="chain1only" filter="pre_sspred_overall" confidence="0" />

            <SSPrediction name="pre_mismatch_probability" confidence="0" cmd="/software/psipred4/runpsipred_single" use_probability="1" mismatch_probability="1" use_svm="0" />
            <MoveBeforeFilter name="mismatch_probability" mover="chain1only" filter="pre_mismatch_probability" confidence="0" />

            <ScorePoseSegmentFromResidueSelectorFilter name="has_sheet" in_context="1" residue_selector="sheetA" scorefxn="sfxn" confidence="0" />
            <ScorePoseSegmentFromResidueSelectorFilter name="was_done" in_context="1" residue_selector="DONE_res" scorefxn="sfxn" confidence="0" />

            <SSShapeComplementarity name="ss_sc_pre" verbose="0" confidence="0" />
            <MoveBeforeFilter name="ss_sc" mover="chain1only" filter="ss_sc_pre" confidence="0"/>

            <Time name="timed"/>
            <TrueFilter name="my_true_filter" />
        </FILTERS>
        <MOVERS>
            <FastDesign name="FastDesign" scorefxn="sfxn_design" repeats="1" task_operations="current,limitchi2,ex1_ex2,restrict_to_interface,restrict_target2repacking,restrict_hotspots2repacking,disallow_GLY,disallow_PRO,PR_monomer_core,upweight_interface,restrict_PRO_GLY,prune_buried_unsats,layer_design_F_boundary_M" batch="false" ramp_down_constraints="false" cartesian="false" bondangle="false" bondlength="false" min_type="dfpmin_armijo_nonmonotone" relaxscript="InterfaceDesign2019" >
                <MoveMap name="MM" >
                        <Chain number="1" chi="true" bb="true" />
                        <Chain number="2" chi="true" bb="false" />
                        <Jump number="1" setting="true" />
                </MoveMap>
            </FastDesign>
            ################################ prevent repacking of rif? ##################################################
            <FastRelax name="FastRelax" scorefxn="sfxn_relax" repeats="1" batch="false" ramp_down_constraints="false" cartesian="false" bondangle="false" bondlength="false" min_type="dfpmin_armijo_nonmonotone" task_operations="current,ex1_ex2,restrict_target_not_interface,limitchi2,prune_buried_unsats" >
                <MoveMap name="MM" >
                    <Chain number="1" chi="true" bb="true" />
                    <Chain number="2" chi="true" bb="false" />
                    <Jump number="1" setting="true" />
                </MoveMap>
            </FastRelax>
            <ParsedProtocol name="go_go_go" mode="sequence">
                <Add filter="gate_filter" />
                Add mover="genProfile" />
                <Add mover="3trp" />
                <Add mover="2met" />
                <Add mover="30_percent_polar" />
                <Add mover="FastDesign" />
                <Add mover="save_output" />
                <Add mover="FastDesign" />
                <Add mover="clear_constraints" />
                <Add mover="FastRelax" />
                <Add mover="add_done" />
                <Add mover="save_output" />
            </ParsedProtocol>

            <GenericMonteCarlo name="try_go_go_go" mover_name="go_go_go" filter_name="my_true_filter" trials="1" />

        </MOVERS>
        <APPLY_TO_POSE>
        </APPLY_TO_POSE>
        <PROTOCOLS>
            <Add filter="timed" />

            <Add mover="remove_done" />
            <Add mover="save_output" />
            <Add mover="try_go_go_go" />
            <Add mover="load_output" />
            <Add filter_name="interface_buried_sasa" />   
            <Add filter_name="ddg" />                     
            <Add filter="ddg_hydrophobic" />              
            <Add filter_name="interface_sc" />            
            <Add filter_name="score_per_res" />          
            Add filter_name="score_per_res_relax" />    
            <Add filter="vbuns5.0_heavy_ball_1.1D" />    
            <Add filter="sbuns5.0_heavy_ball_1.1D" />    
            <Add filter="buns_heavy_ball_1.1D" />    
            <Add filter="hydrophobic_residue_contacts" />
            <Add filter="fa_atr_pocket" />               
            <Add filter="mismatch_probability" />        
            Add filter="sspred_overall" />              
            <Add filter="contact_molecular_surface" />
            <Add filter="contact_patch" />   
            <Add filter="ss_sc" />   
            <Add filter="has_sheet" />
            <Add filter="was_done" />
            <Add filter="timed" />
        </PROTOCOLS>
        <OUTPUT />
    </ROSETTASCRIPTS>
    """.format(
        interface_hydrophobics=scores["interface_hydrophobics"],
        sasa_threshold=sasa_threshold,
    )
    intdes = SingleoutputRosettaScriptsTask(xml)
    designed_ppose = intdes(packed_pose_in.pose.clone())
    new_scores = dict(designed_ppose.pose.scores)
    scores.update(new_scores)
    designed_pose = io.to_pose(designed_ppose)
    for key, value in scores.items():
        pyrosetta.rosetta.core.pose.setPoseExtraScore(designed_pose, key, str(value))
    final_ppose = io.to_packed(designed_pose)
    return final_ppose

In [12]:
%%time
import pyrosetta
from pyrosetta.distributed import cluster
import pyrosetta.distributed.io as io

flags = """
-out:level 300
-corrections::beta_nov16 true
-holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc
-indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5
"""
pyrosetta.distributed.init(" ".join(flags.replace("\n\t", " ").split()))
t = design_binder(
    None,
    **{
        "-s": "/mnt/home/pleung/projects/peptide_binders/r0/peptide_binders/00_thread_targets/decoys/0000/2021.07.12.10.35.38.485279_00431f82398d4a56b6d2e29532614d17.pdb.bz2",
        "sasa_threshold": 0
    }
)

192,195,196
gip
core.select.residue_selector.util: {0} [ WARNING ] Selector name is empty!
core.select.residue_selector.util: {0} [ WARNING ] Selector name is empty!
core.select.residue_selector.util: {0} [ WARNING ] Selector name is empty!
core.select.residue_selector.util: {0} [ WARNING ] Selector name is empty!
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ################ Cloning pose and building neighbor graph ################
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] Ensure that pose is either scored or has update_residue_neighbors() called
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] before using NeighborhoodResidueSelector for maximum performance!
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ##########################################################################
core.select.residue_selector.NeighborhoodResidueSelector: {0} [ WARNING ] ################ Clonin

In [5]:
t.pose.dump_pdb("test.pdb")

True

In [6]:
t.pose.scores

{'9mer': '2.9270942211151123',
 'abego_str': 'XAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAABBBBAAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAAGBBAAAAAAAAAAAAAAAAAAAAOAAAAAAAAAAAAAAAAAAAX',
 'bb_clash': '68.21316528320312',
 'buns_heavy_ball_1.1D': 0.0,
 'chB_length': '20.0',
 'chB_start': '182.0',
 'closure_type': 'strict_remodel',
 'cmsa_AB': '394.10821533203125',
 'cmsa_AC': '236.7537384033203',
 'cmsa_BC': '347.9515075683594',
 'cmsa_int': '613.0968017578125',
 'contact_molecular_surface': 459.23956298828125,
 'contact_patch': 146.7666778564453,
 'ddg': -27.203824996948242,
 'ddg_hydrophobic': -23.278669357299805,
 'disulfide_at': '67,107',
 'docked_helix': '6',
 'dslf_fa13': -0.4751970849763447,
 'dslf_fa13_cart': '-0.8929347395896912',
 'dssp': 'LHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHHHHHH

### Setup dask, set command line options, make tasks and submit to client again to test msd
all filtered stapled helix binders vs all targets

In [ ]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from glob import glob
import logging
import pwd
from pyrosetta.distributed.cluster.core import PyRosettaCluster


print("run the following from your local terminal:")
print(
    f"ssh -L 8000:localhost:8787 {pwd.getpwuid(os.getuid()).pw_name}@{socket.gethostname()}"
)


def create_tasks(scaffolds, targets, options):
    with open(scaffolds, "r") as f:
        for file in f:
            for target in targets:
                tasks = {"options": "-corrections::beta_nov16 true"}
                tasks["extra_options"] = options
                tasks["-s"] = file.rstrip()
                tasks["target"] = target
                yield tasks


logging.basicConfig(level=logging.INFO)
scaffolds = os.path.join(os.getcwd(), "00_inputs/closed.list")
targets = glob(os.path.join(os.getcwd(), "00_inputs/targets/0/*.pdb"))


options = {
    "-out:level": "300",
    "-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
    "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
#     "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
#     "-indexed_structure_store:fragment_store": "/net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5",
}

output_path = os.path.join(os.getcwd(), "00_thread_targets")

if __name__ == "__main__":
    # configure SLURM cluster as a context manager
    with SLURMCluster(
        cores=1,
        processes=1,
        job_cpu=1,
        memory="4GB",
        queue="long",
        walltime="23:30:00",
        death_timeout=120,
        local_directory="$TMPDIR/dask",
        log_directory="/mnt/home/pleung/logs/slurm_logs",
        extra=["--lifetime", "23h", "--lifetime-stagger", "4m"],
    ) as cluster:
        print(cluster.job_script())
        # scale between 1-100 workers,
        cluster.adapt(
            minimum=1,
            maximum=100,
            wait_count=400,  # Number of consecutive times that a worker should be suggested for removal it is removed
            interval="5s",  # Time between checks
        )
        # setup a client to interact with the cluster as a context manager
        with Client(cluster) as client:
            print(client)
            PyRosettaCluster(
                tasks=create_tasks(scaffolds, targets, options),
                client=client,
                scratch_dir=output_path,
                output_path=output_path,
            ).distribute(protocols=[thread_target])

run the following from your local terminal:
ssh -L 8000:localhost:8787 pleung@dig148
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /mnt/home/pleung/logs/slurm_logs/dask-worker-%J.err
#SBATCH -o /mnt/home/pleung/logs/slurm_logs/dask-worker-%J.out
#SBATCH -p long
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=4G
#SBATCH -t 23:30:00

JOB_ID=${SLURM_JOB_ID%;*}

/home/pleung/.conda/envs/phil/bin/python -m distributed.cli.dask_worker tcp://172.16.131.32:44355 --nthreads 1 --memory-limit 3.73GiB --name name --nanny --death-timeout 120 --local-directory $TMPDIR/dask --lifetime 23h --lifetime-stagger 4m

<Client: 'tcp://172.16.131.32:44355' processes=0 threads=0, memory=0 B>


INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'options': '-run:constant_seed 1 -multithreading:total_threads 1', 'extra_options': '-mute all', 'set_logging_handler': 'interactive', 'silent': True}
INFO:pyrosetta.rosetta:Found rosetta database at: /home/pleung/.conda/envs/phil/lib/python3.8/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.12+release.ed6a5560506cfd327d4a6a3e2c9b0c9f6f4a6535 2021-03-26T16:09:25] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [25]:
for (
    i,
    p,
) in enumerate(tpposes):
    p.pose.dump_pdb(f"{i}.pdb")
    print(
        i,
        p.pose.scores["interface_hydrophobics"],
        p.pose.scores["interface_hydrophobic_count"],
    )

0  0.0
1 201,208 2.0
2 208,212 2.0
3 208,212 2.0
4 201 1.0
5 212 1.0
6 208,212 2.0
7 208 1.0
8 201,212 2.0
9 201,208,212 3.0
10 201,208 2.0
11  0.0
12 208,212 2.0
13 201,208,212 3.0
14 201 1.0
15 212 1.0
